In [1]:
%load_ext watermark
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.colors
from matplotlib.colors import LinearSegmentedColormap, ListedColormap
import seaborn as sns

import session_config
import reports
import geospatial
import userdisplay as disp
from myst_nb import glue
from IPython.display import display, Markdown

# available data
surveys = session_config.collect_survey_data()

# Use cases

How each stake holder interprets the data is dependent on the use case. Here we present the answers to some of the most common questions that can be answered using the report or the methods desccribed in this document.

## Is their more now than there was before ?

__How do you know ?__

## Where is the cleanest beach ?

__How do you find the cleanest locations ?__

## How much of the items are single use plastics ?

__Is their more now than there was before ?__

## How much of the items can you identify ?

__Is their more now than there was before ?__

## What objects are common to all cantons ?

## What is the chance of finding X ?